In [12]:
import pandas as pd
from db import Company, CoalCompanyPerformance
import re

In [2]:
minerba_df = pd.read_csv('minerba.csv')

In [3]:
companies = Company.select()
for company in companies:
    pass

In [4]:
ccp = CoalCompanyPerformance.select()
for c in ccp:
    pass

In [19]:
minerba_df['nama_usaha_cleaned'] = minerba_df['nama_usaha'].str.lower().str.strip()

In [32]:
minerba_df.columns

Index(['Unnamed: 0', 'objectid', 'pulau', 'pejabat', 'id_prov', 'nama_prov',
       'id_kab', 'nama_kab', 'jenis_izin', 'badan_usaha', 'nama_usaha',
       'kode_wiup', 'sk_iup', 'tgl_berlaku', 'tgl_akhir', 'kegiatan',
       'luas_sk', 'komoditas', 'kode_golongan', 'kode_jnskom', 'cnc',
       'generasi', 'kode_wil', 'lokasi', 'geometry', 'nama_usaha_cleaned'],
      dtype='object')

In [41]:
for c in (CoalCompanyPerformance
          .select(CoalCompanyPerformance, Company)
          .join(Company)):
  
  company_name = c.company.name
  company_name_cleaned = re.sub(r'\b(PT|Tbk)\b', '', company_name).lower().strip()

  if (company_name_cleaned in minerba_df['nama_usaha_cleaned'].values) and \
      (c.mining_permit is None):
      
      # Get data from dataframe
      mask = minerba_df['nama_usaha_cleaned'] == company_name_cleaned
      jenis_izin, luas_sk, geometry = minerba_df[mask][['jenis_izin', 'luas_sk', 'geometry']].values[0]

      # Update the record in DB
      c.mining_permit = jenis_izin
      c.area = luas_sk
      c.area_geometry = geometry
      # c.save()  # Save changes!

      print(f"Updated {c.id} \t {c.year} \t {c.company.name}: permit={jenis_izin}, area={luas_sk}")